In [1]:
options(warn=-1)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,509065,27.2,1095180,58.5,608525,32.5
Vcells,974035,7.5,8388608,64.0,1606620,12.3


In [49]:
folder <- "analysis1201"

In [38]:
library(xgboost)
library(readr)
library(stringr)
library(caret)
library(car)
library(DMwR)
library(Hmisc)
library(plyr)
library(dplyr)
library(tidyr)
library(data.table)
library(DescTools)
library(Matrix)
library(glmnet)
library(plotrix)
library(Ckmeans.1d.dp)

set.seed(0)
setwd("C:/Users/Jeane/MSDS/Proj2")

# load data
df_train = read_csv("C:/Users/Jeane/MSDS/Proj2/airbnb/train_users_2.csv")
df_test = read_csv("C:/Users/Jeane/MSDS/Proj2/airbnb/test_users.csv")
age_gender_bkts <- fread("C:/Users/Jeane/MSDS/Proj2/airbnb/age_gender_bkts.csv", data.table=F)
countries <- fread("C:/Users/Jeane/MSDS/Proj2/airbnb/countries.csv", data.table=F)
sample_submission_NDF <- fread("C:/Users/Jeane/MSDS/Proj2/airbnb/sample_submission_NDF.csv", data.table=F)
sessions <- fread("C:/Users/Jeane/MSDS/Proj2/airbnb/sessions.csv", data.table=F)

dir.create(paste0("submit"))


dir.create(paste0("cache/", folder))
dir.create(paste0("cache/", folder, "/valid"))
dir.create(paste0("cache/", folder, "/test"))
dir.create(paste0("submit/", folder))

# Load workspace from data engineering

In [ ]:
#load("dataengineering_wrkspc.RData")
#Takes up a lot of memor space
#Load RDS files as needed

In [3]:
df_all_feats<-readRDS("cache/df_all_feats.RData")
df_all<-readRDS("cache/df_all.RData")

In [4]:
summary(df_all)

      id            date_account_created   dac_year          dac_month        
 Length:275547      Min.   :2010-01-01   Length:275547      Length:275547     
 Class :character   1st Qu.:2013-04-03   Class :character   Class :character  
 Mode  :character   Median :2014-01-03   Mode  :character   Mode  :character  
                    Mean   :2013-09-26                                        
                    3rd Qu.:2014-06-18                                        
                    Max.   :2014-09-30                                        
                                                                              
   dac_day          timestamp_first_active date_first_booking  
 Length:275547      Min.   :2.009e+13      Min.   :2010-01-02  
 Class :character   1st Qu.:2.013e+13      1st Qu.:2012-12-02  
 Mode  :character   Median :2.014e+13      Median :2013-09-11  
                    Mean   :2.013e+13      Mean   :2013-07-04  
                    3rd Qu.:2.014e+13      3rd Q

In [5]:
summary(df_all_feats)

      id              feature              value          
 Length:50545479    Length:50545479    Min.   :     -349  
 Class :character   Class :character   1st Qu.:        0  
 Mode  :character   Mode  :character   Median :       98  
                                       Mean   :   687072  
                                       3rd Qu.:    10559  
                                       Max.   :192860976  
                                       NA's   :5382783    

# Remove age NA's 

In [6]:
Xage_all <- subset(df_all_feats, is.na(value)==F)

In [7]:
Xage_all <- subset(Xage_all, feature %nin% c("age"))
Xage_all <- subset(Xage_all, feature %nin% c("age_cln"))
Xage_all <- subset(Xage_all, feature %nin% c("age_cln2"))

In [8]:
Xage_all$feature_name <- Xage_all$feature
Xage_all$feature <- as.numeric(as.factor(Xage_all$feature))

In [9]:
Xage <- subset(Xage_all, id %in% subset(df_all, is.na(age)==F)$id)
Xage_test <- subset(Xage_all, id %in% subset(df_all, is.na(age)==T)$id)
Xage_all <- rbind(Xage, Xage_test)

In [13]:
dim(Xage)
dim(Xage_test)
dim(Xage_all)

[1] 27232327        5

[1] 17611227        5

[1] 44843554        4

In [61]:
tail(Xage_test)

id,feature,value,feature_name,id_num
gpusl6ppgf,3365,0,language_levenshtein_distance.US,54413
kofaz2kh70,3365,0,language_levenshtein_distance.US,67282
u7lv3glv6y,3365,0,language_levenshtein_distance.US,98304
8yvhec201j,3365,0,language_levenshtein_distance.US,29324
fa6260ziny,3365,0,language_levenshtein_distance.US,49717
87k0fy4ugm,3365,0,language_levenshtein_distance.US,26853


In [10]:
Xage_all_feature <- Xage_all[!duplicated(Xage_all$feature), c("feature", "feature_name")]
Xage_all_feature <- Xage_all_feature[order(Xage_all_feature$feature),]

In [11]:
Xage$id_num <- as.numeric(as.factor(Xage$id))
Xage_test$id_num <- as.numeric(as.factor(Xage_test$id))

In [12]:
Xage_id <- Xage %>% distinct(id_num)

In [20]:
dim(Xage_id)
head(Xage_id)

[1] 158681      1

id_num
35494
19664
50772
36178
96310
1597


In [23]:
Xage_id <- data.frame(id = as.character(Xage_id$id_num), id_num = Xage_id$id_num)

In [24]:
Xage_id <- dplyr::left_join(Xage_id, df_all[c("id", "age")], by = "id")

In [25]:
Xage_id <- Xage_id %>%
  dplyr::arrange(id_num)

In [27]:
dim(Xage_id)
head(Xage_id)
summary(Xage_id)

[1] 158681      3

id,id_num,age
1,1,NA
2,2,NA
3,3,NA
4,4,NA
5,5,NA
6,6,NA


      id                id_num            age        
 Length:158681      Min.   :     1   Min.   : NA     
 Class :character   1st Qu.: 39671   1st Qu.: NA     
 Mode  :character   Median : 79341   Median : NA     
                    Mean   : 79341   Mean   :NaN     
                    3rd Qu.:119011   3rd Qu.: NA     
                    Max.   :158681   Max.   : NA     
                                     NA's   :158681  

In [30]:
summary(Xage_test)

      id               feature         value          feature_name      
 Length:17611227    Min.   :   1   Min.   :    -167   Length:17611227   
 Class :character   1st Qu.:1505   1st Qu.:       0   Class :character  
 Mode  :character   Median :3135   Median :      62   Mode  :character  
                    Mean   :2475   Mean   :  743681                     
                    3rd Qu.:3357   3rd Qu.:   10115                     
                    Max.   :3437   Max.   :26944728                     
     id_num      
 Min.   :     1  
 1st Qu.: 29176  
 Median : 58482  
 Mean   : 58466  
 3rd Qu.: 87754  
 Max.   :116866  

In [32]:
y_age <- Xage_id$age

In [34]:
Xage_test_id <- Xage_test %>% distinct(id_num)

In [35]:
summary(Xage_test_id)

     id_num      
 Min.   :     1  
 1st Qu.: 29217  
 Median : 58434  
 Mean   : 58434  
 3rd Qu.: 87650  
 Max.   :116866  

In [36]:
Xage_test_id <- data.frame(id = as.character(Xage_test_id$id_num), id_num = Xage_test_id$id_num)
Xage_test_id <- Xage_test_id %>%
  dplyr::arrange(id_num)
y_age_test <- rep(NA, nrow(Xage_test_id))

In [37]:
common_feature <- dplyr::intersect(unique(Xage$feature), unique(Xage_test$feature))

In [38]:
Xage <- na.omit(Xage)
Xage <- subset(Xage, feature %in% common_feature)
Xage_sp <- sparseMatrix(i = Xage$id_num,
                      j = Xage$feature,
                      x = Xage$value)
dim(Xage_sp)

[1] 158681   3437

In [58]:
summary(y_age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
     NA      NA      NA     NaN      NA      NA  158681 

In [39]:
Xage_test <- na.omit(Xage_test)
Xage_test <- subset(Xage_test, feature %in% common_feature)
Xage_test_sp <- sparseMatrix(i = Xage_test$id_num,
                           j = Xage_test$feature,
                           x = Xage_test$value)
dim(Xage_test_sp)

dXage <- xgb.DMatrix(Xage_sp, label = y_age, missing = -99999)
dXage_test <- xgb.DMatrix(Xage_test_sp, missing = -99999)

[1] 116866   3437

In [56]:
str(Xage_sp)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:25886758] 0 7 9 10 11 17 21 22 23 25 ...
  ..@ p       : int [1:3438] 0 56199 118162 123828 124119 125539 126143 126927 127005 127097 ...
  ..@ Dim     : int [1:2] 158681 3437
  ..@ Dimnames:List of 2
  .. ..$ : NULL
  .. ..$ : NULL
  ..@ x       : num [1:25886758] 3 77 4 14 6 2 3 42 1 79 ...
  ..@ factors : list()


In [53]:
str(Xage_test_sp)


Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:17611085] 2 4 6 7 9 10 20 21 22 27 ...
  ..@ p       : int [1:3438] 0 44903 91355 93266 93332 93626 93748 93948 93960 94050 ...
  ..@ Dim     : int [1:2] 116866 3437
  ..@ Dimnames:List of 2
  .. ..$ : NULL
  .. ..$ : NULL
  ..@ x       : num [1:17611085] 15 5 171 1 1 1 10 1 1 27 ...
  ..@ factors : list()


In [14]:
age_x_all <- subset(df_all_feats, is.na(value)==F)

age_x_all <- subset(age_x_all, feature %nin% c("age"))
age_x_all <- subset(age_x_all, feature %nin% c("age_cln"))
age_x_all <- subset(age_x_all, feature %nin% c("age_cln2"))

age_x_all$feature_name <- age_x_all$feature
age_x_all$feature <- as.numeric(as.factor(age_x_all$feature))
age_x <- subset(age_x_all, id %in% subset(df_all, is.na(age_cln2)==F)$id)
age_x_test <- subset(age_x_all, id %in% subset(df_all, is.na(age_cln2)==T)$id)
age_x_all <- rbind(age_x, age_x_test)

age_x_all_feature <- age_x_all[!duplicated(age_x_all$feature), c("feature", "feature_name")]
age_x_all_feature <- age_x_all_feature[order(age_x_all_feature$feature),]

age_x$id_num <- as.numeric(as.factor(age_x$id))
age_x_test$id_num <- as.numeric(as.factor(age_x_test$id))

In [15]:
age_x_id <- age_x %>% distinct(id_num, .keep_all= TRUE)
age_x_id <- data.frame(id = age_x_id$id, id_num = age_x_id$id_num)
age_x_id <- dplyr::left_join(age_x_id, df_all[c("id", "age_cln2")], by = "id")
age_x_id <- age_x_id %>%
  dplyr::arrange(id_num)
age_y <- age_x_id$age_cln2

age_x_test_id <- age_x_test %>% distinct(id_num, .keep_all= TRUE)
age_x_test_id <- data.frame(id = age_x_test_id$id, id_num = age_x_test_id$id_num)
age_x_test_id <- age_x_test_id %>%
  dplyr::arrange(id_num)
age_y_test <- rep(NA, nrow(age_x_test_id))


common_feature <- dplyr::intersect(unique(age_x$feature), unique(age_x_test$feature))

age_x <- na.omit(age_x)
age_x <- subset(age_x, feature %in% common_feature)
age_x_sp <- sparseMatrix(i = age_x$id_num,
                      j = age_x$feature,
                      x = age_x$value)
dim(age_x_sp)

age_x_test <- na.omit(age_x_test)
age_x_test <- subset(age_x_test, feature %in% common_feature)
age_x_test_sp <- sparseMatrix(i = age_x_test$id_num,
                           j = age_x_test$feature,
                           x = age_x_test$value)
dim(age_x_test_sp)

dage_x <- xgb.DMatrix(age_x_sp, label = age_y, missing = -99999)
dage_x_test <- xgb.DMatrix(age_x_test_sp, missing = -99999)

[1] 158681   3437

[1] 116866   3437

In [16]:
param <- list("objective" = "reg:linear",
                "eval_metric" = "rmse",
                "eta" = 0.1,
                "max_depth" = 6,
                "subsample" = 0.7,
                "colsample_bytree" = 0.3,
                "alpha" = 1.0,
                "nthread" = 24)
  
xgb <- xgboost(data = dage_x, 
                 param = param,
                 nround = 200)

[1]	train-rmse:33.844414 
[2]	train-rmse:30.926428 
[3]	train-rmse:28.348259 
[4]	train-rmse:26.061132 
[5]	train-rmse:24.054253 
[6]	train-rmse:22.287609 
[7]	train-rmse:20.743185 
[8]	train-rmse:19.407734 
[9]	train-rmse:18.247290 
[10]	train-rmse:17.250404 
[11]	train-rmse:16.395561 
[12]	train-rmse:15.660428 
[13]	train-rmse:15.037786 
[14]	train-rmse:14.517586 
[15]	train-rmse:14.072237 
[16]	train-rmse:13.700367 
[17]	train-rmse:13.388674 
[18]	train-rmse:13.137529 
[19]	train-rmse:12.925071 
[20]	train-rmse:12.744998 
[21]	train-rmse:12.600475 
[22]	train-rmse:12.475808 
[23]	train-rmse:12.373174 
[24]	train-rmse:12.289108 
[25]	train-rmse:12.216737 
[26]	train-rmse:12.156615 
[27]	train-rmse:12.105117 
[28]	train-rmse:12.065570 
[29]	train-rmse:12.028770 
[30]	train-rmse:11.997495 
[31]	train-rmse:11.972548 
[32]	train-rmse:11.951260 
[33]	train-rmse:11.933535 
[34]	train-rmse:11.917153 
[35]	train-rmse:11.903411 
[36]	train-rmse:11.888631 
[37]	train-rmse:11.875985 
[38]	train

In [36]:
importance_matrix <- xgb.importance(model = xgb)
print(importance_matrix)
impm <- dataframe(importance)
xgb.plot.importance(importance_matrix = importance_matrix[0:15])

      Feature         Gain        Cover    Frequency
   1:    3294 4.039799e-02 1.138537e-02 0.0021244954
   2:    3278 4.013628e-02 1.528630e-02 0.0029742936
   3:    3407 2.348432e-02 3.981781e-03 0.0010622477
   4:    3328 1.736438e-02 7.929362e-03 0.0015933716
   5:    3329 1.424686e-02 3.347801e-03 0.0009560229
  ---                                               
1337:     443 9.732342e-07 9.003681e-08 0.0001062248
1338:     107 6.255789e-07 3.751534e-08 0.0001062248
1339:      49 6.195710e-07 2.250920e-08 0.0001062248
1340:      51 5.111475e-07 2.250920e-08 0.0001062248
1341:      68 3.438985e-07 3.751534e-08 0.0002124495


ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): unable to start png() device


plot without title

In [ ]:
(gg <- xgb.ggplot.importance(importance_matrix, measure = "Frequency", rel_to_first = TRUE))
gg + ggplot2::ylab("Frequency")

In [ ]:
capabilities()

In [ ]:
library(WVPlots)

In [ ]:
plot_fit_trajectory(xgb$evaluation_log)

In [42]:
age_x_stack <- data.frame()
age_x_test_stack <- data.frame()
age_y_df <- data.frame(age_pred = age_y)
age_y_df <- mutate(age_y_df,
                id = unique(age_x_id$id))
age_y_df <- age_y_df[c("id", "age_pred")]
age_x_stack <- bind_rows(age_x_stack,
                    age_y_df)

In [43]:
age_y_test_ <- predict(xgb, dage_x_test)
age_y_test_df <- data.frame(age_pred = age_y_test_)
age_y_test_df <- mutate(age_y_test_df,
                   id = unique(age_x_test_id$id))
age_y_test_df <- age_y_test_df[c("id", "age_pred")]
age_x_test_stack <- bind_rows(age_x_test_stack,
                         age_y_test_df)

In [45]:
age_x_test_stack <- age_x_test_stack %>%
  group_by(id) %>%
  summarise_all(funs(mean))

In [46]:
age_x_stack <- melt.data.table(as.data.table(age_x_stack))
age_x_stack <- data.frame(age_x_stack)
names(age_x_stack) <- c("id", "feature", "value")

In [47]:
age_x_test_stack <- melt.data.table(as.data.table(age_x_test_stack))
age_x_test_stack <- data.frame(age_x_test_stack)
names(age_x_test_stack) <- c("id", "feature", "value")

In [50]:
saveRDS(age_x_stack, paste0("cache/",folder,"/age_stack.RData"))
saveRDS(age_x_test_stack, paste0("cache/",folder,"/age_test_stack.RData"))